In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tqdm as tqdm
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
import gc
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

In [ ]:
# legacy function to load unglobbed dataset
def load_image(image_path):
  input_image = tf.image.decode_image(tf.io.read_file(image_path))
  if input_image.shape[-1] == 4:
    input_image = input_image[...,:-1]
  input_image = tf.cast(input_image, tf.float32)
  return tf.expand_dims(input_image,0)

In [ ]:
# directory_path = "/content/gdrive/My Drive/planegan/sample_output_3/"
# image_lst = []
# for file in os.listdir(directory_path):
#  add_image = load_image(os.path.join(directory_path,file))
#  image_lst.append(add_image)

In [ ]:
#function to load a globbed image dataset
def load_custom_image_set(dataroot,directorylevels):
    #image parse function
    def parse_image(filename):
        parts = tf.strings.split(filename, os.sep)
        label = parts[-2]
        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = image[...,:-1]
        image = (image * 255)
        return tf.expand_dims(image,0), label

    data_root = pathlib.Path(dataroot)
    if directorylevels == 1:
      list_ds = tf.data.Dataset.list_files(str(data_root/'*/*'))
    if directorylevels == 2:
      list_ds = tf.data.Dataset.list_files(str(data_root/'*/*/*'))
    images_ds = list_ds.map(parse_image)
    #https://stackoverflow.com/questions/70535683/extract-data-from-tensorflow-dataset-e-g-to-numpy
    images = np.asarray(list(images_ds.map(lambda x, y: x)))
    return images

In [ ]:
# load globbed directory image data
image_path = "/content/gdrive/My Drive/planegan/glob_dir/"
directorylevels = 1
image_lst = load_custom_image_set(image_path,directorylevels)

In [ ]:
model = hub.load(SAVED_MODEL_PATH)

In [ ]:
start = time.time()
counter = 0
for idx in range(len(image_lst)):
  fake_image = model(image_lst[idx])
  fake_image = tf.squeeze(fake_image)
  image = np.asarray(fake_image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.figure(figsize=(21,30))
  plt.imshow(image)
  plt.axis('off')
  filename = '/content/gdrive/My Drive/planegan/sample_output_3a/generated_plot_%05d.png' % (counter+1)
  plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
  plt.close()
  counter += 1
  print("Time Taken: %f" % (time.time() - start))



Time Taken: 30.834551
Time Taken: 60.913351


In [ ]:
from google.colab import runtime
runtime.unassign()